## Laser Calibration ```gcode``` examples.

Some code to 'calibrate' laser settings to a material.

In [5]:
# Imports and autoreload for development
%load_ext autoreload
%autoreload 2
import gcode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import grbl
cnc = grbl.Grbl(port="/dev/ttyUSB0")

## Point Calibration.

Vary laser power & dwell times in a grid with 5mmx5mm spacing

In [26]:
# Create an empty GCode program.
prog = gcode.GCode(machine=cnc)
prog.G21()
prog.G90()
prog.G92(X=0, Y=0)
prog.G1(F=200)
prog.G0(F=200)
spacing = 5
for y, dwell in enumerate([0.5, 1, 2, 5, 10]):
    for x, power in enumerate([100, 150, 200, 250]):
        prog.G0(X=x*spacing, Y=y*spacing)
        prog.M3(S=power)
        prog.G1(X=x*spacing, Y=y*spacing)
        prog.G4(P=dwell)
        prog.M5()

In [22]:
cnc.status

'<Hold:0|WPos:39.996,95.004,0.000|Bf:15,114|FS:0,0|Ov:100,100,100>'

In [23]:
cnc.reset()

(0, ['', "Grbl 1.1g ['$' for help]", 'ok'])

In [24]:
cnc.status

'<Idle|WPos:39.996,95.004,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [25]:
cnc.run(prog)

115.87804841995239

# Line Calibration

Vary feed rate, power and dynamic powering.

In [7]:
def init(**kwargs):
    """
    Basic Machine Init.
    """
    prog = gcode.GCode(**kwargs)
    prog.G21()
    prog.G90()
    prog.G92(X=0, Y=0)
    prog.G1(F=200)
    prog.G0(F=200)
    return prog

# Coordinate Example

In [8]:
# Create an empty GCode program with the above function, some saner defaults than an empty program.
prog = init(machine=cnc)
# Line spacing to use.
line_spacing=5
# Generate points for a 10mm line
line_points = gcode.hline(Xf=10)
# For each of the given powers
for p, power in enumerate([100, 150, 175, 200, 255]):
    # Use Absolute coordinates
    prog.G90()
    # Go to a the start of a row, each power has its own row.
    prog.G0(X=0, Y=p*line_spacing)
    # Relative coordinates for each drawn line.
    prog.G91()
    # Using and not using dynamic power.
    for d, dynamic_power in enumerate([True, False]):
        # For each of the given feed rates
        for f, feed in enumerate([60, 120, 180, 300]): 
                line = gcode.Line(points=line_points, power=power, feed=feed, dynamic_power=dynamic_power)
                prog+=line
prog.G90()
prog.G0(X=0, Y=0)

In [9]:
cnc.status

'<Check|WPos:0.000,0.000,0.000|Bf:15,127|FS:0,0>'

In [12]:
cnc.cmd("$C")

['ok', '[MSG:Disabled]', 'ok', '', "Grbl 1.1g ['$' for help]"]

In [13]:
cnc.run(prog)

.....

267.92051005363464